# Full energy optimization demo

This notebook shows how to build optimization inputs from the Newcastle Urban Case
study data (metadata, weather, and stochastic occupant-driven profiles) and then
plot the resulting consumption and comfort trajectories.

---


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure repository code is importable
import sys
from pathlib import Path

def find_repo_root(start=None):
    path = Path(start or Path.cwd()).resolve()
    for candidate in [path] + list(path.parents):
        if (candidate / '.git').exists():
            return candidate
    return path

repo_root = find_repo_root()
sys.path.append(str(repo_root / 'Codes' / 'sourcecode'))

from RC_Optimization import optimize_full_energy_system, build_tariff


## Build inputs

- Load dwelling metadata (R, C, g, occupants) from the updated Newcastle Urban Case CSV
- Load the winter weather time series (outdoor temperature and solar irradiance)
- Sample daily hot-water demand, appliance consumption, and EV availability profiles
  from the stochastic CREST-style CSVs based on occupant count
- Build time-aligned tariffs and set-point schedules for optimization


In [ ]:
# Paths to prepared datasets (update if your repo layout differs)
data_root = repo_root / 'Input data'
meta_path = data_root / 'Newcastle_Urban_Case_meta_updated.csv'
weather_path = data_root / 'NEcase_20_21_t2m_ssrd_30min.csv'
profile_root = data_root / 'Stochastic_Demands'

# Profile column mapping (update after inspecting the stochastic demand files)
profile_column_map = {
    'hotwater': ['hotwater'],
    'appliance': ['appliance'],
    'ev_availability': ['ev_availability'],
}

# Analysis horizon
step = '30min'
steps_per_day = int(pd.Timedelta('1D') / pd.Timedelta(step))

# Load metadata
meta = pd.read_csv(meta_path)

# Inspect metadata columns
print('Metadata columns:', list(meta.columns))
display(meta.head())

# Explicit column names (update these to match your CSVs)
meta_columns = {
    'occupants': 'occupants',
    'R1': 'R1',
    'C1': 'C1',
    'g': 'g',
}

weather_columns = {
    'time': 'time',
    't2m': 't2m',
    'ssrd': 'ssrd',
}



def _load_daily_samples_from_array(data, steps_per_day):
    if data.ndim == 1:
        data = data.reshape(-1, 1)

    if data.shape[0] == steps_per_day:
        return data, 1
    if data.shape[1] == steps_per_day:
        return data, 0

    flat = data.reshape(-1)
    if flat.size % steps_per_day == 0:
        reshaped = flat.reshape(-1, steps_per_day)
        return reshaped, 0

    raise ValueError(f'Cannot infer daily samples from array with shape {data.shape}')


def select_profile_by_index(samples, axis, idx, steps_per_day):
    if axis == 1:
        profile = samples[:, idx]
    else:
        profile = samples[idx, :]
    if len(profile) < steps_per_day:
        profile = np.pad(profile, (0, steps_per_day - len(profile)), mode='edge')
    elif len(profile) > steps_per_day:
        profile = profile[:steps_per_day]
    return profile


def find_occ_profile_file(root, occ):
    root = Path(root)
    csv_paths = sorted(root.glob('*.csv'))
    if not csv_paths:
        raise FileNotFoundError(f'No CSV files found in {root}')

    occ = int(occ)
    for path in csv_paths:
        name = path.stem.lower()
        if f'occ{occ}' in name or f'occ_{occ}' in name or f'occ-{occ}' in name:
            return path

    # Fallback: if there are 5 files, map occupant 1-5 to sorted order
    if len(csv_paths) == 5 and 1 <= occ <= 5:
        return csv_paths[occ - 1]

    raise FileNotFoundError(
        f'No occupant-specific profile CSV found for occ={occ} in {root}. '
        'Rename the files to include "occ{n}" or update the matching logic.'
    )


occ_col = meta_columns['occupants']
r_col = meta_columns['R1']
c_col = meta_columns['C1']
g_col = meta_columns['g']

# Load weather
weather = pd.read_csv(weather_path)

# Inspect weather columns
print('Weather columns:', list(weather.columns))
display(weather.head())

time_col = weather_columns['time']
if time_col in weather.columns:
    weather[time_col] = pd.to_datetime(weather[time_col])
    weather = weather.set_index(time_col)
else:
    weather.index = pd.date_range('2020-01-01', periods=len(weather), freq=step)

weather = weather.sort_index()

# Pick temperature and irradiance columns
temp_col = weather_columns['t2m']
solar_col = weather_columns['ssrd']
if temp_col not in weather.columns:
    raise KeyError(f'Column {temp_col} not found in weather CSV.')
if solar_col not in weather.columns:
    raise KeyError(f'Column {solar_col} not found in weather CSV.')

Tout = weather[temp_col].astype(float)
if Tout.median() > 100:
    Tout = Tout - 273.15

# Convert ssrd to W/m^2 if needed
S = weather[solar_col].astype(float)

dt_seconds = (weather.index[1] - weather.index[0]).total_seconds()
if S.max() > 2000:
    S = S / dt_seconds

# Select a shorter window for demo runs (change n_days or start_date as needed)
start_date = weather.index[0].normalize()
n_days = 2
end_date = start_date + pd.Timedelta(days=n_days)
window = weather.loc[start_date:end_date - pd.Timedelta(step)]
Tout = Tout.loc[window.index].to_numpy()
S = S.loc[window.index].to_numpy()

# Tariff aligned with weather window
Tariff_start = window.index[0]
tariff = build_tariff(Tariff_start, n_days=n_days, step=step, type='cosy')
if not tariff.index.equals(window.index):
    tariff = tariff.reindex(window.index, method='ffill')

n_steps = len(tariff)

# Heating set-points
hours = tariff.index.hour + tariff.index.minute / 60.0
comfort_setpoint = np.where((hours >= 6) & (hours < 9) | (hours >= 17) & (hours < 22), 21.0, 17.0)
flex_setpoint = np.where((hours >= 6) & (hours < 8) | (hours >= 18) & (hours < 21), 20.0, 16.0)
setpoint_sequences = [comfort_setpoint, flex_setpoint]

# Helpers to sample daily profiles
rng = np.random.default_rng(42)
profile_keywords = profile_column_map


def sample_stochastic_profiles(path, n_days, steps_per_day, rng):
    df = pd.read_csv(path)
    profile_data = {key: [] for key in profile_keywords}
    chosen_indices = []

    for _ in range(n_days):
        run_idx = int(rng.integers(1, 1001))
        chosen_indices.append(run_idx)
        resolved = []
        min_days = None
        for profile_type in profile_keywords:
            col = f"run_{run_idx}_{profile_type}"
            if col not in df.columns:
                raise KeyError(f"Expected column {col} in {path}.")
            series = pd.to_numeric(df[col], errors='coerce').fillna(0).to_numpy()
            samples, axis = _load_daily_samples_from_array(series, steps_per_day)
            resolved.append((profile_type, samples, axis))
            n_candidates = samples.shape[1] if axis == 1 else samples.shape[0]
            min_days = n_candidates if min_days is None else min(min_days, n_candidates)

        if min_days is None or min_days == 0:
            raise ValueError(f'No daily samples found in {path} for run {run_idx}.')

        day_idx = int(rng.integers(0, min_days))

        for profile_type, samples, axis in resolved:
            profile = select_profile_by_index(samples, axis, day_idx, steps_per_day)
            profile_data[profile_type].append(profile)

    for profile_type in profile_keywords:
        profile_data[profile_type] = np.concatenate(profile_data[profile_type])

    chosen_by_type = {profile_type: chosen_indices for profile_type in profile_keywords}

    return profile_data, chosen_by_type


# Build per-dwelling inputs based on occupant count

# Inspect a sample stochastic profile file to confirm column names
try:
    sample_profile_path = find_occ_profile_file(profile_root, 1)
    sample_profile = pd.read_csv(sample_profile_path)
    print(f'Stochastic profile sample file: {sample_profile_path.name}')
    print('Profile columns:', list(sample_profile.columns))
    display(sample_profile.head())
except FileNotFoundError as exc:
    print(exc)
max_dwellings = None  # set to an int for quicker runs in the demo

if max_dwellings:
    meta_iter = meta.head(max_dwellings).copy()
else:
    meta_iter = meta

dwelling_inputs = {}
for idx, row in meta_iter.iterrows():
    occ = row[occ_col]
    profile_path = find_occ_profile_file(profile_root, occ)
    profiles, chosen = sample_stochastic_profiles(profile_path, n_days, steps_per_day, rng)

    appliance_profile = profiles['appliance']

    # Appliance profile: assume kW if values are small, otherwise W
    if np.nanmax(appliance_profile) < 50:
        base_electric = appliance_profile * 1000
    else:
        base_electric = appliance_profile

    dwelling_inputs[idx] = {
        'occ': occ,
        'meta': row,
        'hw_demand': profiles['hotwater'],
        'base_electric': base_electric,
        'ev_availability': profiles['ev_availability'],
        'samples': chosen,
        'profile_path': str(profile_path),
    }

print(f'Prepared inputs for {len(dwelling_inputs)} dwellings over {n_days} days.')






## Run the optimization

We evaluate both set-point schedules in one call. The solver returns the optimal solution for each schedule and highlights the least-cost option via the `best_key` and `best_result` entries.

Hot-water handling can be configured with `hw_mode`:
- `boiler_only` for HHP dwellings without storage (all DHW from the boiler).
- `hp_storage` for mHP dwellings with a storage tank following the additional constraints.
- `hybrid_direct` to retain the previous direct split between heat pump and boiler.

The storage case now tracks the stored **volume** at a fixed 55°C tank temperature per the provided formulation.


In [ ]:
# Select a dwelling to optimize
selected_dwelling_id = next(iter(dwelling_inputs))
selected = dwelling_inputs[selected_dwelling_id]

# Thermal model parameters from metadata
params = {
    'R1': float(selected['meta'][r_col]) if r_col else 1 / 200,
    'C1': float(selected['meta'][c_col]) if c_col else 3e7,
    'g': float(selected['meta'][g_col]) if g_col else 10.0,
    'dt': dt_seconds,
    'T0': 20.0,
    'tol': 1.0,
    'COP': 3.5,
    'etaB': 0.9,
    'Qhp_max': 5e3,   # W
    'Qbo_max': 20e3,  # W
}

hw_demand = selected['hw_demand']
base_electric = selected['base_electric']

# EV parameters (availability from profile; assume full battery at start)
ev_capacity_kwh = 80.0

ev_params = {
    'ev_capacity': ev_capacity_kwh * 1000,  # Wh
    'ev_target': 0.5 * ev_capacity_kwh * 1000,
    'ev_charge_max': 11e3,  # W
    'ev_availability': selected['ev_availability'],
    'eta_ev_charge': 0.95,
    'travel_energy': np.zeros(n_steps),
}

# Hot water configuration: switch to 'boiler_only' for HHP without storage,
# or keep 'hp_storage' to mimic the mHP case with a cylinder.
hw_params = {
    'hw_mode': 'hp_storage',
    'V_stor': 0.2,       # m^3 storage volume capacity
    'V_stor_init': 0.12, # initial stored volume (m^3)
    'T_mains': 10.0,
    'T_hw_supply': 40.0,
}

results = optimize_full_energy_system(
    tariff=tariff,
    Tout=Tout,
    S=S,
    setpoint_sequences=setpoint_sequences,
    hw_demand=hw_demand,
    base_electric=base_electric,
    day_ahead=True,
    **params,
    **ev_params,
    **hw_params,
)

best_key = results.get('best_key', next(k for k in results if k.startswith('schedule_')))
best = results['best_result']['results'] if 'best_result' in results else results[best_key]['results']
best_cost = results['best_result']['cost'] if 'best_result' in results else results[best_key]['cost']
print(f"Best schedule: {best_key}, total cost = {best_cost:.2f}")

best.head()



## Plot consumption profiles

The figure shows:

- Electricity and gas prices
- Indoor temperature versus comfort band
- Space and hot water heat delivery
- Storage tank water temperature when `hw_mode='hp_storage'`
- EV state of charge
- Aggregate electricity and gas consumption profiles


In [ ]:
use_storage = np.isfinite(best['T_stor']).any()

n_rows = 6 if use_storage else 5
fig, axs = plt.subplots(n_rows, 1, figsize=(12, 18 if use_storage else 16), sharex=True)

axs[0].plot(tariff.index, tariff['elec_price'], label='Electricity price (p/kWh)')
axs[0].plot(tariff.index, tariff['gas_price'], label='Gas price (p/kWh)')
axs[0].legend(); axs[0].set_ylabel('Tariff'); axs[0].grid(True)

axs[1].plot(best.index, best['Tin'], label='Indoor temp')
axs[1].plot(best.index, best['T_set'], '--', label='Set-point')
axs[1].fill_between(best.index, best['T_low'], best['T_high'], color='lightblue', alpha=0.3, label='Comfort band')
axs[1].legend(); axs[1].set_ylabel('Temperature (°C)'); axs[1].grid(True)

axs[2].plot(best.index, best['Q_hp_space']/1000, label='HP space (kW)')
axs[2].plot(best.index, best['Q_bo_space']/1000, label='Boiler space (kW)')
axs[2].plot(best.index, best['Q_hp_hw']/1000, label='HP hot water (kW)')
axs[2].plot(best.index, best['Q_bo_hw']/1000, label='Boiler hot water (kW)')
axs[2].legend(); axs[2].set_ylabel('Thermal output (kW)'); axs[2].grid(True)

if use_storage:
    tank_ax = axs[3]
    vol_ax = tank_ax.twinx()
    tank_ax.plot(best.index, best['T_stor'], label='Storage water temp (°C)')
    tank_ax.axhline(hw_params.get('T_mains', np.nan), color='k', linestyle=':', label='Mains temp')
    vol_ax.plot(best.index, best['V_stor'], color='tab:orange', label='Stored volume (m³)')
    vol_ax.bar(best.index, hw_demand, width=0.02, alpha=0.3, color='tab:green', label='Hot water demand (m³)')
    tank_ax.legend(loc='upper left'); vol_ax.legend(loc='upper right')
    tank_ax.set_ylabel('Tank temp (°C)')
    vol_ax.set_ylabel('Stored volume / demand (m³)')
    tank_ax.grid(True)

ev_ax = axs[3] if not use_storage else axs[4]
plugin_series = pd.Series(ev_params['ev_availability'], index=best.index)
plugged = plugin_series > 0.5
if plugged.any():
    spans = []
    start = None
    for ts, avail in plugged.items():
        if avail and start is None:
            start = ts
        elif not avail and start is not None:
            spans.append((start, ts))
            start = None
    if start is not None:
        spans.append((start, plugin_series.index[-1]))
    for i, (s, e) in enumerate(spans):
        ev_ax.axvspan(s, e, color='gray', alpha=0.2, label='Plug-in window' if i == 0 else None)

ev_ax.plot(best.index, best['ev_soc'], label='EV SOC (kWh)')
ev_ax.bar(best.index, best['P_ev_charge'], width=0.02, alpha=0.4, label='EV charge (kW)')
ev_ax.legend(); ev_ax.set_ylabel('EV metrics'); ev_ax.grid(True)

# Aggregate consumption
heat_pump_elec = (best['Q_hp_space'] + best['Q_hp_hw']) / params['COP'] / 1000  # kW
other_elec = base_electric / 1000  # kW
ev_elec = best['P_ev_charge']  # already kW
gas_input = (best['Q_bo_space'] + best['Q_bo_hw']) / params['etaB'] / 1000  # kW (thermal -> fuel)

agg_ax = axs[4] if not use_storage else axs[5]
agg_ax.plot(best.index, heat_pump_elec + other_elec + ev_elec, label='Electric load (kW)')
agg_ax.plot(best.index, gas_input, label='Gas input (kW)')
agg_ax.legend(); agg_ax.set_ylabel('Power (kW)'); agg_ax.grid(True)

plt.xlabel('Time')
plt.tight_layout()
plt.show()
